In [1]:
%load_ext autoreload
%autoreload 2

import logging
import os
import sys

l = logging.getLogger()
lh = logging.StreamHandler(sys.stdout)
l.addHandler(lh)
l.setLevel(logging.INFO)

import torch

import experiment
import stats

# Set up some logging if you want. The harness will
# log to its own logger. You can capture those logs and
# write them to file, via the root logger.
# Point it wherever you want
#LOG_DIR = os.path.join(os.curdir, "logs")

# If you want to save logs to file
#root_logger = logging.getLogger()
#root_logger.addHandler(logging.FileHandler())

# cuda="0" for the first GPU in CUDA's "PCI_BUS_ID" order.
# See: https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
# For single GPU systems, just specify "0". nvidia-smi can be used
# to find your GPU PCI order.
# Can also supply cuda=None for CPU-based run
# Will return a torch.device or None as the last return value.
cfg = experiment.get_config(cuda="0")

import attr
class MyCoProc(experiment.CoProc):
    @attr.s(auto_attribs=True)
    class MyUserData(stats.UserData):
        hello: str
        blah: float
        
        def render(self):
            return {
                       "hello": self.hello,
                       "blah": self.blah
                   }
        
            
    def __init__(self, cfg):
        self.cfg = cfg
        
    def forward(self, brain_data, loss_history):
        batch_size = brain_data[0].shape[0]
        stim = torch.ones(batch_size, cfg.stim_dim) * 0.01
        
        if cfg.cuda is not None:
            stim.cuda(cfg.cuda)
            
        return stim
    
    def feedback(self, actuals, targets, loss_history):
        if (loss_history.eidx % 20) == 0 and loss_history.eidx > 0:
            stop = True
        else:
            stop = False
            
        is_validation = False
        update_task_loss = True
        user_data = MyCoProc.MyUserData("hello", 1.2)
        
        return stop, update_task_loss, is_validation, user_data
    
    def finish(self, loss_history):
        return False
        
my_coproc = MyCoProc(cfg)

In [2]:
# Start running experiment; this and the next cell can be killed with
#  ctrl+c, and resumed with the cell below.

my_experiment = experiment.stage(my_coproc, cfg)

In [3]:
loss_history = my_experiment.run()

Hello 0 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 1 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 2 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 3 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 4 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 5 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 6 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 7 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 8 task_loss: 0.020536 pct_recov: 0.009 class_separation: -0.145 user: {'hello': 'hello', 'blah': 1.2}
Hello 9 task_loss: 0.020536 

In [4]:
# Continue running experiment
loss_history = my_experiment.run()

In [ ]:
# Show me how I did, make pretty graphs, etc.
experiment.score(my_experiment.loss_history)